### Weekly Market Updates for the Treasury Team

##### User needs to be integrated to EVDS API for this dashboard. Bloomberg API is not needed for the following cells
##### You can obtain EVDS API key by signing to: https://evds2.tcmb.gov.tr/index.php?/evds/login

In [23]:
# import packages
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline

from datetime import timedelta
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from evds import evdsAPI

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import datetime

#### Enter a date range

In [3]:
# user input for date range
start_date = "01-09-2021"
end_date = "27-04-2022"

##### Enter your EVDS API key

In [ ]:
evds = evdsAPI('xxxx')

### Total Assets / Foreign Assets of CBRT


In [19]:
data = evds.get_data(['TP.AB.A01','TP.AB.A02'], startdate= start_date, enddate= end_date)
data.rename(
    columns=({ 'TP_AB_A01': 'Aktif', 'TP_AB_A02': 'Dis_Varlik'}), 
    inplace=True,
)
data['ratio'] = data['Dis_Varlik']/data['Aktif']

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=data.Tarih, y=data.Aktif,
                        mode = 'lines', name='Total Assets'),secondary_y=False)
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Dis_Varlik,
                        mode = 'lines', name='Foreign Assets',
                        line = dict(color='firebrick', width=1)),secondary_y=False)

fig.add_trace(
    go.Bar(
        y=data.ratio,
        x=data.Tarih,
        name='Foreign/Assets Ratio'),
    secondary_y=True)
fig.update_layout(yaxis2_range=[0.75,2])

fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure1.html', auto_open=True)

### Deposit of State in CBRT

In [21]:
data = evds.get_data(['TP.AB.A25'], startdate= start_date , enddate= end_date)
data.rename(
    columns=({ 'TP_AB_A25': 'Kamu_Mevduati'}), 
    inplace=True,
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Kamu_Mevduati,
                        mode = 'lines', name='State Deposit (mln TRY)'))

fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure2.html', auto_open=True)

### Gross and Net Reserves of CBRT

In [26]:
from datetime import datetime
a = pd.to_datetime(start_date)
c = datetime.strftime(a,'%m/%d/%y')
import datetime
b = datetime.datetime.strptime(c, '%m/%d/%y').strftime('%d/%m/%y')
e = pd.to_datetime(b)
d = e - timedelta(days=1500)
from datetime import datetime
f = datetime.strftime(d,'%d-%m-%y')
year = f[:6] + '20' + f[-2:]

data = evds.get_data(['TP.AB.A02', 'TP.AB.A10',
                     'TP.DK.USD.A.YTL'], startdate= year, enddate= end_date)

data2 = evds.get_data(['TP.DOVVARNC.K15'], startdate= year, enddate= end_date)

data.rename(
    columns=({'TP.DOVVARNC.K15': 'Outstanding_Swaps', 'TP.AB.A02': 'Foreign_Assets',
             'TP.AB.A10': 'Total_Foreign_Liabilities',
             'TP.DK.USD.A.YTL': 'USDTRY_Rate'}), 
    inplace=True,
)

def get_month(data):
    if data.Tarih[3:4] == '0':
        return data.Tarih[4:5]
    elif data.Tarih[3:4] != '0': 
        return data.Tarih[3:5]

data['month'] = data.apply(get_month, axis=1)    
data['year'] = data['Tarih'].str[6:10]
data['matching_date'] = data['year'] + '-' + data['month']

data['Tarih'] = pd.to_datetime(data.Tarih, format='%d-%m-%Y')
data = data.drop(['month', 'year'], axis = 1)


gross_reserves = evds.get_data(['TP.AB.C1', 'TP.AB.C2'], startdate= year, enddate= end_date)
gross_reserves['total_gross_reserves'] = gross_reserves['TP_AB_C1']+gross_reserves['TP_AB_C2']
gross_reserves['Date'] = pd.to_datetime(gross_reserves.Tarih, format='%d-%m-%Y')    
gross_reserves.set_index('Date').resample('D').ffill().reset_index()


data3 = pd.merge(data,
                 gross_reserves,
                 left_on = 'Tarih', 
                 right_on = 'Date', 
                 how='right')


data2['Tarih'] = data2.rename(columns=({'Tarih': 'matching_date'}),
                             inplace = True)
data2['matching_date'] = data2['matching_date'].astype('string')


data2['next_month_integer'] = data2['matching_date'].str[5:]
data2['next_month_integer'] = data2['next_month_integer'].astype('int')
def next_month_swaps(data):
    if data.next_month_integer == 12:
        return 1
    elif data.next_month_integer != 12:
        return data.next_month_integer+1

data2['next_year_integer'] = data2['matching_date'].str[:4].astype('int')
def next_year_swaps(data):
    if data.next_month_integer == 12:
        return data.next_year_integer+1
    elif data.next_month_integer != 12:
        return data.next_year_integer  
data2['next_year_integer'] = data2.apply(next_year_swaps, axis=1)  
data2['next_month_integer'] = data2.apply(next_month_swaps, axis=1)  

data2['next_month_integer'] = data2['next_month_integer'].astype('string')
data2['next_year_integer'] = data2['next_year_integer'].astype('string')

data2['matching_date'] = data2['next_year_integer'] + '-' + data2['next_month_integer']
data2 = data2.drop(['Tarih', 'next_month_integer', 'next_year_integer'], axis = 1)


result = pd.merge(data3,
                 data2,
                 on='matching_date', 
                 how='inner')

result.dropna(subset = ["TP_AB_A02"], inplace=True)

result['Net_Reserves'] = (((result['TP_AB_A02']-result['TP_AB_A10'])/result['TP_DK_USD_A_YTL'])/1000)+result['TP_DOVVARNC_K15']


## Visualizing data

fig = go.Figure()
fig.add_trace(go.Scatter(x=result.Date, y=result.Net_Reserves,
                        mode = 'lines', name='Net Reserves'))
fig.add_trace(go.Scatter(x=result.Date, y=result.total_gross_reserves,
                        mode = 'lines', name='Gross Reserves',
                        line = dict(color='firebrick', width=1)))


fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure3.html', auto_open=True)

### Corporate and Retail TRY Loans

In [27]:
data = evds.get_data(['TP.KTF17', 'TP.BFTUKKRE.L027','TP.KTFTUK', 'TP.BFTUKKRE.L004'], startdate= start_date, enddate= end_date)

data.rename(columns=({'TP_KTF17': 'Corporate_Rate', 'TP_BFTUKKRE_L027': 'Corporate_Volume','TP_KTFTUK': 'Retail_Rate','TP_BFTUKKRE_L004': 'Retail_Volume'}), inplace=True,)

## Visualizing data

### Corporate ###
fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating next plot
fig.add_trace(
    go.Bar(
        y=data.Corporate_Volume,
        x=data.Tarih,
        name='Corporate Volume'),
    secondary_y=False)

# creating first plot
fig.add_trace(
    go.Scatter(
        y=data.Corporate_Rate,
        x=data.Tarih,
        name='Corporate Rate'),
    secondary_y=True,
)
fig.update_layout(yaxis_range=[200000000,500000000])

pio.write_html(fig, file='figure4.html', auto_open=True)

### Retail ###
fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating next plot
fig.add_trace(
    go.Bar(
        y=data.Retail_Volume,
        x=data.Tarih,
        name='Retail Volume'),
    secondary_y=False)

# creating first plot
fig.add_trace(
    go.Scatter(
        y=data.Retail_Rate,
        x=data.Tarih,
        name='Retail Rate'),
    secondary_y=True,
)
fig.update_layout(yaxis_range=[400000000,800000000])

pio.write_html(fig, file='figure5.html', auto_open=True)

### XU100 Perfomance vs CPI


In [ ]:
data = evds.get_data(['TP.MK.F.BILESIK'], startdate= start_date, enddate= end_date)

from datetime import datetime
a = pd.to_datetime(start_date)
c = datetime.strftime(a,'%m/%d/%y')
import datetime
b = datetime.datetime.strptime(c, '%m/%d/%y').strftime('%d/%m/%y')
e = pd.to_datetime(b)
d = e - timedelta(days=30)

data2 = evds.get_data(['TP.FG.J0'], startdate= d, enddate= end_date)

data.rename(
    columns=({'TP.MK.F.BILESIK': 'XU100'}), 
    inplace=True,
)

data2.rename(
    columns=({'TP.FG.J0': 'CPI'}), 
    inplace=True,
)

def get_month(data):
    if data.Tarih[3:4] == '0':
        return data.Tarih[4:5]
    elif data.Tarih[3:4] != '0': 
        return data.Tarih[3:5]

data['month'] = data.apply(get_month, axis=1)    
data['year'] = data['Tarih'].str[6:10]
data['matching_date'] = data['year'] + '-' + data['month']

data['Tarih'] = pd.to_datetime(data.Tarih, format='%d-%m-%Y')
data = data.drop(['month', 'year'], axis = 1)



data2['Tarih'] = data2.rename(columns=({'Tarih': 'matching_date'}),
                             inplace = True)
data2['matching_date'] = data2['matching_date'].astype('string')


data2['next_month_integer'] = data2['matching_date'].str[5:]
data2['next_month_integer'] = data2['next_month_integer'].astype('int')
def next_month_swaps(data):
    if data.next_month_integer == 12:
        return 1
    elif data.next_month_integer != 12:
        return data.next_month_integer+1

data2['next_year_integer'] = data2['matching_date'].str[:4].astype('int')
def next_year_swaps(data):
    if data.next_month_integer == 12:
        return data.next_year_integer+1
    elif data.next_month_integer != 12:
        return data.next_year_integer  
    
data2['next_year_integer'] = data2.apply(next_year_swaps, axis=1)  
data2['next_month_integer'] = data2.apply(next_month_swaps, axis=1)  

data2['next_month_integer'] = data2['next_month_integer'].astype('string')
data2['next_year_integer'] = data2['next_year_integer'].astype('string')

data2['matching_date'] = data2['next_year_integer'] + '-' + data2['next_month_integer']
data2 = data2.drop(['Tarih', 'next_month_integer', 'next_year_integer'], axis = 1)


data_aggr = pd.merge(data,
                 data2,
                 on='matching_date', 
                 how='left')

data_aggr['XU100CPI_Ratio'] = data_aggr['TP_MK_F_BILESIK'] / data_aggr['TP_FG_J0']


fig = go.Figure()
fig.add_trace(go.Scatter(x=data_aggr.Tarih, y=data_aggr.XU100CPI_Ratio,
                        mode = 'lines', line=dict(color="#d62728"), name='XU100/CPI Ratio'))

fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure6.html', auto_open=True)


### XU Banking Performance vs CPI

In [28]:
data = evds.get_data(['TP.MK.F.MALI'], startdate= start_date, enddate= end_date)

from datetime import datetime
a = pd.to_datetime(start_date)
c = datetime.strftime(a,'%m/%d/%y')
import datetime
b = datetime.datetime.strptime(c, '%m/%d/%y').strftime('%d/%m/%y')
e = pd.to_datetime(b)
d = e - timedelta(days=30)

data2 = evds.get_data(['TP.FG.J0'], startdate= d, enddate= end_date)

data.rename(
    columns=({'TP.MK.F.MALI': 'XU100'}), 
    inplace=True,
)

data2.rename(
    columns=({'TP.FG.J0': 'CPI'}), 
    inplace=True,
)

def get_month(data):
    if data.Tarih[3:4] == '0':
        return data.Tarih[4:5]
    elif data.Tarih[3:4] != '0': 
        return data.Tarih[3:5]

data['month'] = data.apply(get_month, axis=1)    
data['year'] = data['Tarih'].str[6:10]
data['matching_date'] = data['year'] + '-' + data['month']

data['Tarih'] = pd.to_datetime(data.Tarih, format='%d-%m-%Y')
data = data.drop(['month', 'year'], axis = 1)



data2['Tarih'] = data2.rename(columns=({'Tarih': 'matching_date'}),
                             inplace = True)
data2['matching_date'] = data2['matching_date'].astype('string')


data2['next_month_integer'] = data2['matching_date'].str[5:]
data2['next_month_integer'] = data2['next_month_integer'].astype('int')
def next_month_swaps(data):
    if data.next_month_integer == 12:
        return 1
    elif data.next_month_integer != 12:
        return data.next_month_integer+1

data2['next_year_integer'] = data2['matching_date'].str[:4].astype('int')
def next_year_swaps(data):
    if data.next_month_integer == 12:
        return data.next_year_integer+1
    elif data.next_month_integer != 12:
        return data.next_year_integer  
    
data2['next_year_integer'] = data2.apply(next_year_swaps, axis=1)  
data2['next_month_integer'] = data2.apply(next_month_swaps, axis=1)  

data2['next_month_integer'] = data2['next_month_integer'].astype('string')
data2['next_year_integer'] = data2['next_year_integer'].astype('string')

data2['matching_date'] = data2['next_year_integer'] + '-' + data2['next_month_integer']
data2 = data2.drop(['Tarih', 'next_month_integer', 'next_year_integer'], axis = 1)


data_aggr = pd.merge(data,
                 data2,
                 on='matching_date', 
                 how='left')

data_aggr['XU100BankCPI_Ratio'] = data_aggr['TP_MK_F_MALI'] / data_aggr['TP_FG_J0']


fig = go.Figure()
fig.add_trace(go.Scatter(x=data_aggr.Tarih, y=data_aggr.XU100BankCPI_Ratio,
                        mode = 'lines', line=dict(color="#d62728"), name='XUBanks/CPI Ratio'))

fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure7.html', auto_open=True)

### TRY Deposits 

In [29]:
data = evds.get_data(['TP.TRY.MT06', 'TP.MEVDUAT.I004','TP.MEVDUAT.I023'], startdate= start_date, enddate= end_date)

data.rename(columns=({'TP_TRY_MT06': 'TRY_Rate', 'TP_MEVDUAT_I004': 'Demand_deposit_volume','TP_MEVDUAT_I023': 'Time_deposit_volume'}), inplace=True,)


fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating next plot
fig.add_trace(go.Bar(
    name = 'Demand Deposit',
    x = data.Tarih,
    y = data.Demand_deposit_volume
   ),secondary_y=False)
fig.add_trace(go.Bar(
    name = 'Time Deposit',
    x = data.Tarih,
    y = data.Time_deposit_volume
   ),
             secondary_y = False)
 
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        y=data.TRY_Rate,
        x=data.Tarih,
        name='TRY Rate',line=dict(color="Black", width=2)),
    secondary_y=True,
)
fig.update_layout(yaxis_range=[200000000,2500000000])

pio.write_html(fig, file='figure8.html', auto_open=True)

### USD Deposits


In [ ]:
data = evds.get_data(['TP.USD.MT06', 'TP.YPMEVD.M33','TP.YPMEVD.M37'], startdate= start_date, enddate= end_date)

data.rename(columns=({'TP_USD_MT06': 'USD_Rate', 'TP_YPMEVD_M33': 'Demand_deposit_volume','TP_YPMEVD_M37': 'Time_deposit_volume'}), inplace=True,)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating next plot
fig.add_trace(go.Bar(
    name = 'Demand Deposit',
    x = data.Tarih,
    y = data.Demand_deposit_volume
   ),secondary_y=False)
fig.add_trace(go.Bar(
    name = 'Time Deposit',
    x = data.Tarih,
    y = data.Time_deposit_volume
   ),
             secondary_y = False)
 
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        y=data.USD_Rate,
        x=data.Tarih,
        name='USD Rate',line=dict(color="Black", width=2)),
    secondary_y=True,
)
fig.update_layout(yaxis_range=[40000,170000])

pio.write_html(fig, file='figure9.html', auto_open=True)

### EUR Deposits

In [30]:
data = evds.get_data(['TP.EUR.MT06', 'TP.YPMEVD.M46','TP.YPMEVD.M50'], startdate= start_date, enddate= end_date)

data.rename(columns=({'TP_EUR_MT06': 'EUR_Rate', 'TP_YPMEVD_M46': 'Demand_deposit_volume','TP_YPMEVD_M50': 'Time_deposit_volume'}), inplace=True,)

data2 = evds.get_data(['TP.DK.USD.A.YTL', 'TP.DK.EUR.A.YTL'], startdate= start_date, enddate= end_date)
data2.rename(columns=({'TP_DK_USD_A_YTL': 'USDTRY', 'TP_DK_EUR_A_YTL': 'EURTRY'}), inplace=True,)
data2['EURUSD'] = data2['EURTRY']/data2['USDTRY']

data_aggr = pd.merge(data,
                 data2,
                 on='Tarih', 
                 how='left')
data_aggr['Demand_deposit_volume_USD'] = data_aggr['Demand_deposit_volume']/data_aggr['EURUSD']
data_aggr['Time_deposit_volume_USD'] = data_aggr['Time_deposit_volume']/data_aggr['EURUSD']


## Visualizing data

fig = make_subplots(specs=[[{"secondary_y": True}]])

# creating next plot
fig.add_trace(go.Bar(
    name = 'Demand Deposit (in USD)',
    x = data_aggr.Tarih,
    y = data_aggr.Demand_deposit_volume_USD
   ),secondary_y=False)
fig.add_trace(go.Bar(
    name = 'Time Deposit (in USD)',
    x = data_aggr.Tarih,
    y = data_aggr.Time_deposit_volume_USD
   ),
             secondary_y = False)
 
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        y=data_aggr.EUR_Rate,
        x=data_aggr.Tarih,
        name='EUR Rate',line=dict(color="Black", width=2)),
    secondary_y=True,
)
fig.update_layout(yaxis_range=[20000,120000])

pio.write_html(fig, file='figure10.html', auto_open=True)

### Foreign Ownership in securities, repos etc.

In [31]:
data = evds.get_data(['TP.MKNETHAR.M7', 'TP.MKNETHAR.M8',
                     'TP.MKNETHAR.M9'], startdate= start_date, enddate= end_date)


data.rename(
    columns=({'TP_MKNETHAR_M7': 'Stock_Change', 'TP_MKNETHAR_M8': 'Debt_Securities',
             'TP_MKNETHAR_M9' : 'Reverse_Repo'}), 
    inplace=True,
)


data = data.drop(columns=['YEARWEEK'])

## Visualizing data

fig = go.Figure()
fig.add_trace(go.Scatter(
        x= data['Tarih'], y = data['Stock_Change'],
        name = 'Stock_Change',
        mode = 'lines', line=dict(width=0.1,color='lightgreen'),
        fill = 'tozeroy'))
fig.add_trace(go.Scatter(
        x= data['Tarih'], y = data['Debt_Securities'],
        name = 'Debt_Securities',
        mode = 'lines', line=dict(width=0.1, color='darkred'),
        fill = 'tozeroy'))
fig.add_trace(go.Scatter(
        x= data['Tarih'], y = data['Reverse_Repo'],
        name = 'Reverse_Repo',
        mode = 'lines', line=dict(width=0.5, color='yellow'),
        fill = 'tozeroy'))

pio.write_html(fig, file='figure11.html', auto_open=True)

### Major Owners of TRY Denominated Debt Securities

In [32]:
data = evds.get_data(['TP.DIBSYAZDEG.S122', 'TP.DIBSYAZDEG.S1234',
                     'TP.DIBSYAZDEG.S125','TP.DIBSYAZDEG.S126',
                     'TP.DIBSYAZDEG.S128','TP.DIBSYAZDEG.S129'], startdate= start_date, enddate= end_date)


data.rename(
    columns=({'TP_DIBSYAZDEG_S122': 'Banks', 'TP_DIBSYAZDEG_S1234': 'Mutual_Funds',
             'TP_DIBSYAZDEG_S125' : 'Other_Financial_Mediators',
             'TP_DIBSYAZDEG_S126' : 'Financial_Mediators',
             'TP_DIBSYAZDEG_S128' : 'Insurance_Funds',
             'TP_DIBSYAZDEG_S129' : 'Retirement_Funds'}), 
    inplace=True,
)

data['Other_Funds'] = data['Mutual_Funds']+data['Other_Financial_Mediators']+data['Financial_Mediators']+data['Insurance_Funds']+data['Retirement_Funds']

## Visualizing data
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=data.Tarih, y=data.Banks,
                        mode = 'lines', name='Banks'),secondary_y = False,)
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Other_Funds,
                        mode = 'lines', name='Other Funds',
                        line = dict(color='firebrick', width=1)),secondary_y = True,)



fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=True, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

fig.update_yaxes(title_text="Banks", secondary_y=False)
fig.update_yaxes(title_text="Other Funds", secondary_y=True)

pio.write_html(fig, file='figure12.html', auto_open=True)

### Base Money (TRY)

In [ ]:
data = evds.get_data(['TP.AB.A02', 'TP.AB.A03',
                     'TP.AB.A08','TP.AB.A10',
                     'TP.AB.A13','TP.AB.A21',
                     'TP.AB.A22','TP.AB.A23'], startdate= start_date, enddate= end_date)


data.rename(
    columns=({'TP_AB_A02': 'Dıs_Varlık', 
             'TP_AB_A03' : 'Ic_Varlık',
             'TP_AB_A08' : 'Degerleme_Hesabı',
             'TP_AB_A10' : 'Toplam_Doviz_Yukumlulukleri',
             'TP_AB_A13' : 'Kamu_ve_Diger_Doviz_Mevduatı',
             'TP_AB_A21' : 'Fon_Hesapları',
             'TP_AB_A22' : 'Banka_Dısı_Kesim_Mevduatı',
             'TP_AB_A23' : 'Diger_Merkez_Bankası_Parası'}), 
    inplace=True,
)

data['Net_Ic_Varlıklar'] = data['Ic_Varlık']+data['Degerleme_Hesabı']-data['Diger_Merkez_Bankası_Parası']-data['Fon_Hesapları']-data['Banka_Dısı_Kesim_Mevduatı']-data['Kamu_ve_Diger_Doviz_Mevduatı']
data['Net_Dıs_Varlıklar'] = data['Dıs_Varlık']-data['Toplam_Doviz_Yukumlulukleri']
data['Para_Tabanı'] = data['Net_Ic_Varlıklar'] + data['Net_Dıs_Varlıklar']


## Visualizing data
fig = go.Figure()
fig.add_trace(go.Scatter(
        x= data['Tarih'], y = data['Para_Tabanı'],
        name = 'Base Money (TRY)',
        mode = 'lines', line=dict(width=0.1,color='darkred'),
        fill = 'tozeroy'))

pio.write_html(fig, file='figure13.html', auto_open=True)

### Major Owners of TRY Denominated Short Term Debt Securities

In [ ]:
data = evds.get_data(['TP.DIBSVADE.C1.S122', 'TP.DIBSVADE.C1.S1234',
                     'TP.DIBSVADE.C1.S125','TP.DIBSVADE.C1.S126',
                     'TP.DIBSVADE.C1.S128','TP.DIBSVADE.C1.S129'], startdate= start_date, enddate= end_date)


data.rename(
    columns=({'TP_DIBSVADE_C1_S122': 'Banks', 'TP_DIBSVADE_C1_S1234': 'Mutual_Funds',
             'TP_DIBSVADE_C1_S125' : 'Other_Financial_Mediators',
             'TP_DIBSVADE_C1_S126' : 'Financial_Mediators',
             'TP_DIBSVADE_C1_S128' : 'Insurance_Funds',
             'TP_DIBSVADE_C1_S129' : 'Retirement_Funds'}), 
    inplace=True,
)

data['Other_Funds'] = data['Mutual_Funds']+data['Other_Financial_Mediators']+data['Financial_Mediators']+data['Insurance_Funds']+data['Retirement_Funds']

## Visualizing data
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Banks,
                        mode = 'lines', name='Banks'))
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Other_Funds,
                        mode = 'lines', name='Other Funds',
                        line = dict(color='firebrick', width=1)))


fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=True, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure14.html', auto_open=True)

### Major Owners of TRY Denominated Long Term Debt Securities

In [33]:
data = evds.get_data(['TP.DIBSVADE.C2.S122', 'TP.DIBSVADE.C2.S1234',
                     'TP.DIBSVADE.C2.S125','TP.DIBSVADE.C2.S126',
                     'TP.DIBSVADE.C2.S128','TP.DIBSVADE.C2.S129'], startdate= start_date, enddate= end_date)


data.rename(
    columns=({'TP_DIBSVADE_C2_S122': 'Banks', 'TP_DIBSVADE_C2_S1234': 'Mutual_Funds',
             'TP_DIBSVADE_C2_S125' : 'Other_Financial_Mediators',
             'TP_DIBSVADE_C2_S126' : 'Financial_Mediators',
             'TP_DIBSVADE_C2_S128' : 'Insurance_Funds',
             'TP_DIBSVADE_C2_S129' : 'Retirement_Funds'}), 
    inplace=True,
)

data['Other_Funds'] = data['Mutual_Funds']+data['Other_Financial_Mediators']+data['Financial_Mediators']+data['Insurance_Funds']+data['Retirement_Funds']

## Visualizing data
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Banks,
                        mode = 'lines', name='Banks'))
fig.add_trace(go.Scatter(x=data.Tarih, y=data.Other_Funds,
                        mode = 'lines', name='Other Funds',
                        line = dict(color='firebrick', width=1)))


fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=True, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure15.html', auto_open=True)


#### FX Deposit of State in CBRT

In [34]:
data = evds.get_data(['TP.AB.A13'], startdate= start_date , enddate= end_date)
data.rename(
    columns=({ 'TP_AB_A13': 'Kamu_Mevduati'}), 
    inplace=True,
)

data2 = evds.get_data(['TP.DK.USD.A.YTL'], startdate= start_date, enddate= end_date)
data2.rename(columns=({'TP_DK_USD_A_YTL': 'USDTRY'}), inplace=True,)

data_aggr = pd.merge(data,
                 data2,
                 on='Tarih', 
                 how='left')
data_aggr['State_Deposit'] = data_aggr['Kamu_Mevduati']/data_aggr['USDTRY']

## Visualizing data
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_aggr.Tarih, y=data_aggr.State_Deposit,
                        mode = 'lines', name='State FX Deposit(mln USD)'))

fig.update_layout(
xaxis=dict(
showline=True, showgrid=False, showticklabels=True,
linecolor='rgb(204, 204, 204)',
linewidth = 2, ticks ='outside', tickfont =dict(
family= 'Arial', size =12, color='rgb(82, 82, 82)',
),
),
yaxis=dict(showgrid=True, zeroline=False, showline=False,
           showticklabels=True),
autosize = False,
margin = dict(
autoexpand=False, l=100, r=200, t=110, ),
showlegend = True, plot_bgcolor ='white')

pio.write_html(fig, file='figure16.html', auto_open=True)